In [ ]:
cd ..

In [ ]:
import pandas as pd

# Settings

In [ ]:
VOLUME_FILE = "data/raw/gx_volume.csv"
SUBMISSIONS_FILE = "data/raw/submission_template.csv"
PACKAGES_FILE = "data/raw/gx_package.csv"

In [ ]:
volume = pd.read_csv(VOLUME_FILE, index_col=0)
submissions = pd.read_csv(SUBMISSIONS_FILE)

# Create data points

In [ ]:
volume["country_brand"] = volume["country"] + "-" + volume["brand"]
submissions["country_brand"] = submissions["country"] + "-" + submissions["brand"]

In [ ]:
volume_non_submission = volume[~volume["country_brand"].isin(submissions["country_brand"])]
volume_non_submission = volume_non_submission.sort_values(["country", "brand", "month_num"])

In [ ]:
country_brand_post_count = volume_non_submission[volume_non_submission["month_num"] >= 0].groupby("country_brand").size()
idx_post_volume_full = country_brand_post_count[country_brand_post_count == 24].index
volume_non_submission = volume_non_submission[volume_non_submission["country_brand"].isin(idx_post_volume_full)]

In [ ]:
volume_grouped = volume_non_submission.groupby(["country", "brand"])

In [ ]:
country_list = list()
brand_list = list()
x_list = list()
y_list = list()
for group, df in volume_grouped:
    country_list.append(group[0])
    brand_list.append(group[1])
    x_list.append(df[["volume"]][df["month_num"] < 0].values)
    y_list.append(df[["volume"]][df["month_num"] >= 0].values)

# Check outlier brands

In [ ]:
outlier_brands = ["brand_102", "brand_108", "brand_133", "brand_163", "brand_187"]

In [ ]:
for brand in outlier_brands:
    print(brand)
    print("Brand in submissions?", brand in list(submissions["brand"]))
    print("Brand in volume?", brand in list(volume["brand"]))
    print("Brand in volume_non_submission?", brand in list(volume_non_submission["brand"]))
    print("Brand's number of countries in submissions", submissions["country"][submissions["brand"] == brand].nunique())
    print("Brand size in submissions", len(submissions[submissions["brand"] == brand]))
    print("Number of data points with brand:", sum([b == brand for b in brand_list]), "/", len(brand_list))
    print("")

In [ ]:
"brand_163" in brand_list

# Check outlier countrues

In [ ]:
outlier_countries = ["country_8", "country_10", "country_12", "country_16"]

In [ ]:
for country in outlier_countries:
    print(country)
    print("Country in submissions?", country in list(submissions["country"]))
    print("Country in volume?", country in list(volume["country"]))
    print("Country in volume_non_submission?", country in list(volume_non_submission["country"]))
    print("Country's number of brands in submissions", submissions["brand"][submissions["country"] == country].nunique())
    print("Country size in submissions", len(submissions[submissions["country"] == country]))
    print("Number of data points with country:", sum([c == country for c in country_list]), "/", len(country_list))
    print("")

# Check outlier presentations

In [ ]:
packages = pd.read_csv(PACKAGES_FILE, index_col=0)

In [ ]:
volume_packages = volume.merge(packages, on=["country", "brand"], how="left")
submissions_packages = submissions.merge(packages, on=["country", "brand"], how="left")
volume_non_submission_packages = volume_non_submission.merge(packages, on=["country", "brand"], how="left")

In [ ]:
outliers_packages = ["PILL"]

In [ ]:
for package in outliers_packages:
    print(package)
    print("Package in submissions?", package in list(submissions_packages["presentation"]))
    print("Package in volume?", package in list(volume_packages["presentation"]))
    print("Package in volume_non_submission?", package in list(volume_non_submission_packages["presentation"]))
    print("")